# Classifier Model Test

In [ ]:
import sys, os
sys.path.append('../')

In [ ]:
# checkpoint파일에서 가장 큰 epoch인 파일 찾기 ex) best_model_fold_1_epoch_8.pth
import torch 
import os
from model.loader import model_Loader
def max_epoch_selector(checkpoints:list):
    max_epoch, max_epoch_idx = 0, 0
    for idx, checkpoint in enumerate(checkpoints):
        epoch = checkpoint.split('.pth')[0].split('_')[-1]
        if int(epoch) > max_epoch:
            max_epoch, max_epoch_idx = int(epoch), idx
    return max_epoch_idx

def bestmodel_selector(checkpoint_root_dir:str, model_cards:list, idx:int, device = 'cpu'): 
    model_card = model_cards[idx]
    checkpoints = sorted(os.listdir(os.path.join(checkpoint_root_dir, model_card)))
    max_epoch = checkpoints[max_epoch_selector(checkpoints)]
    
    checkpoint_path = os.path.join(checkpoint_root_dir, model_card, max_epoch)
    
    print(model_card)
    
    model_name = model_card.split('_')[0]
    if 'vision-transformer_l_16' in model_card:
        model_type = model_card.split('_')[1] + '_' + model_card.split('_')[2]
    else:        
        model_type = model_card.split('_')[1]
    fold_num = model_card.split('_')[-1].split('-')[0]
    if 'ft' in model_card:
        # convnext_l_fold_1-mask-ft-medsam1
        versions = model_card.split('-')
        # versions를 - 로 합치기 => ex) version = mask-ft-medsam1
        version = '-'.join(versions[1:])
        #version: transformer_default_fold_5-mask-ft-medsam1 -> mask-ft-medsam1
        if model_name == 'swin-transformer':
            version = '-'.join(versions[2:])
    else:
        version = model_card.split('_')[-1].split('-')[1]
    
    if '-' in model_type:
        model_type = model_type.replace('-', '_') 
    
    checkpoint_model = model_Loader(model_name, outlayer_num = 1, type = model_type).to(device)
    model_weights = torch.load(checkpoint_path)['model_state_dict']
    checkpoint_model.load_state_dict(model_weights) 
    print(f"Checkpont Install Complete!! checkpoint_model: {model_name} fold : {fold_num} version: {version}")
    return checkpoint_model, model_name, version, fold_num


In [ ]:
# testloader building 
from lib.dataset import Custom_bus_dataset, JointTransform 
from torch.utils.data import DataLoader
from lib.metric.metrics import multi_classify_metrics, binary_classify_metrics
import pickle 
import pandas as pd
import gc

def memory_release(model):
    gc.collect()
    torch.cuda.empty_cache()
    del model

def dataloader_builder(test_csv_path:str, root_dir:str, input_res = (3, 224, 224), bs_size = 40):
    test_augment_list = JointTransform(
        resize=(input_res[1], input_res[2]),
        horizontal_flip=False,
        vertical_flip=False,
        rotation=0,
        interpolation=False,
    )
    test_dataset = Custom_bus_dataset(
        df = pd.read_csv(test_csv_path),
        root_dir = root_dir,
        joint_transform = test_augment_list 
    )
    test_loader = DataLoader(dataset = test_dataset, batch_size = bs_size, shuffle = False, num_workers=16)

    return test_loader

def test_inference(model, test_loader, version, device):
    all_labels, all_preds, all_probs = [], [], []
    model.eval()
    with torch.no_grad():
        for imgs, masks, labels in test_loader:
            labels = labels.to(device, non_blocking=True).float()
            if version =='mask' or version == 'sammask' or version =='mask-ft-medsam1' or version =='mask-ft-medsam2':
                inputs = imgs.to(device, non_blocking=True)[:, :2, :, :] # for image(2) + mask(1)
                masks = masks.to(device, non_blocking=True)
                outputs = model(torch.concat([inputs, masks], dim=1)) # for image + mask 
            else:
                inputs = imgs.to(device, non_blocking=True)
                outputs = model(inputs) # for image + mask 
            probs = torch.sigmoid(outputs)
            predicted = torch.round(probs)
            
            all_labels.extend(labels.detach().cpu().numpy())
            all_preds.extend(predicted.detach().cpu().numpy())
            all_probs.extend(probs.detach().cpu().numpy())
        
    metrics = binary_classify_metrics(all_labels, all_preds, all_probs, test_on = True)
    return metrics 

def evaluation(model_cards, checkpoint_root_dir, device, test_loader, save_metric_dir):
    for idx, model_card in enumerate(model_cards):
        # Model Setup
        model, model_name, version, fold_num = bestmodel_selector(checkpoint_root_dir, model_cards, idx, device)
        # Inference
        # if os.path.join(save_metric_dir, f'{model_name}_{version}_{fold_num}.pkl') not in os.listdir(save_metric_dir):
        metrics = test_inference(model, test_loader, version, device)
        # memory 초기화
        memory_release(model)
        
        try:
            with open(os.path.join(save_metric_dir, f'{model_name}_{version}_{fold_num}.pkl'), 'wb') as f:
                pickle.dump(metrics, f)
            print("Metrics Save Complete!!", f'{model_name}_{version}_{fold_num}.pkl')
        except MemoryError:
            print(f"Memory Error: {model_name}_{version}_{fold_num}")


In [5]:
#!/usr/bin/env python
import sys, os
sys.path.append('../')
import warnings
import torch 
from lib.seed import set_seed

# 특정 경고 메시지 무시
warnings.filterwarnings("ignore", category=UserWarning, module="torchvision")
warnings.filterwarnings("ignore", category=FutureWarning)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
set_seed(627)
root_dir = '/mnt/hdd/octc/BACKUP/BreastUS'
checkpoint_root_dir = '/mnt/hdd/octc/BACKUP/BreastUS/experiment/checkpoint'
save_metric_dir = '/mnt/hdd/octc/BACKUP/BreastUS/experiment/metrics'
fold_num = 5
seed = 42
outlayer_num = 1

model_cards = sorted(os.listdir(checkpoint_root_dir))

# Original
# origin_test_csv_path = '/mnt/hdd/octc/BACKUP/BreastUS/experiment/test_df.csv'
# origin_test_df = pd.read_csv(origin_test_csv_path)
# origin_test_loader = dataloader_builder(origin_test_csv_path, root_dir, input_res = (3, 224, 224), bs_size = 24)
# origin_model_cards = [model_card for model_card in model_cards if 'origin' == model_card.split('-')[-1]]
# evaluation(model_cards = origin_model_cards, checkpoint_root_dir = checkpoint_root_dir, device = device, test_loader = origin_test_loader, save_metric_dir = save_metric_dir )

# Mask 
# mask_test_csv_path = '/mnt/hdd/octc/BACKUP/BreastUS/experiment/test_df.csv'
# mask_test_df = pd.read_csv(mask_test_csv_path)
# mask_test_loader = dataloader_builder(mask_test_csv_path, root_dir, input_res = (3, 224, 224), bs_size = 24)
# mask_model_cards = [model_card for model_card in model_cards if 'mask' == model_card.split('-')[-1]]
# evaluation(model_cards = mask_model_cards, checkpoint_root_dir = checkpoint_root_dir, device = device, test_loader = mask_test_loader, save_metric_dir = save_metric_dir )

# # SAMMask - Inference
sammask_test_csv_path = '/mnt/hdd/octc/BACKUP/BreastUS/experiment/test_df_bbox.csv'
sammask_test_df = pd.read_csv(sammask_test_csv_path)
sammask_test_loader = dataloader_builder(sammask_test_csv_path, root_dir, input_res = (3, 224, 224), bs_size = 24)
sammask_model_cards = [model_card for model_card in model_cards if 'sammask' == model_card.split('-')[-1]]
evaluation(model_cards = sammask_model_cards, checkpoint_root_dir = checkpoint_root_dir, device = device, test_loader = sammask_test_loader, save_metric_dir = save_metric_dir)

# SAMMask1 - Fine tuning
# sammask_ft_test_csv_path = '/mnt/hdd/octc/BACKUP/BreastUS/experiment/test_df_bbox.csv'
# sammask_ft_test_df = pd.read_csv(sammask_ft_test_csv_path)
# sammask_ft_test_loader = dataloader_builder(sammask_ft_test_csv_path, root_dir, input_res = (3, 224, 224), bs_size = 24)
# sammask_ft_model_cards = [model_card for model_card in model_cards if 'medsam1' == model_card.split('-')[-1]]
# evaluation(model_cards = sammask_ft_model_cards, checkpoint_root_dir = checkpoint_root_dir, device = device, test_loader = sammask_ft_test_loader, save_metric_dir = save_metric_dir)

# SAMMask2 - Fine tuning
# sammask_ft_test_csv_path = '/mnt/hdd/octc/BACKUP/BreastUS/experiment/test_df_bbox.csv'
# sammask_ft_test_df = pd.read_csv(sammask_ft_test_csv_path)
# sammask_ft_test_loader = dataloader_builder(sammask_ft_test_csv_path, root_dir, input_res = (3, 224, 224), bs_size = 24)
# sammask_ft_model_cards = [model_card for model_card in model_cards if 'medsam2' == model_card.split('-')[-1]]
# evaluation(model_cards = sammask_ft_model_cards, checkpoint_root_dir = checkpoint_root_dir, device = device, test_loader = sammask_ft_test_loader, save_metric_dir = save_metric_dir)

Checkpont Install Complete!! checkpoint_model: convnext fold : 1 version: sammask
Metrics Save Complete!! convnext_sammask_1.pkl
convnext_l_fold_2-sammask
Model 'convnext' loaded.
Checkpont Install Complete!! checkpoint_model: convnext fold : 2 version: sammask
Metrics Save Complete!! convnext_sammask_2.pkl
convnext_l_fold_3-sammask
Model 'convnext' loaded.
Checkpont Install Complete!! checkpoint_model: convnext fold : 3 version: sammask
Metrics Save Complete!! convnext_sammask_3.pkl
convnext_l_fold_4-sammask
Model 'convnext' loaded.
Checkpont Install Complete!! checkpoint_model: convnext fold : 4 version: sammask
Metrics Save Complete!! convnext_sammask_4.pkl
convnext_l_fold_5-sammask
Model 'convnext' loaded.
Checkpont Install Complete!! checkpoint_model: convnext fold : 5 version: sammask
Metrics Save Complete!! convnext_sammask_5.pkl
efficient_l_fold_1-sammask
Model 'efficient' loaded.
Checkpont Install Complete!! checkpoint_model: efficient fold : 1 version: sammask
Metrics Save Co

/home/eiden/miniconda3/envs/cv/lib/python3.8/site-packages/torch/functional.py:513: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3609.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Checkpont Install Complete!! checkpoint_model: maxvit fold : 1 version: sammask
Metrics Save Complete!! maxvit_sammask_1.pkl
maxvit_default_fold_2-sammask
Model 'maxvit' loaded.
Checkpont Install Complete!! checkpoint_model: maxvit fold : 2 version: sammask
Metrics Save Complete!! maxvit_sammask_2.pkl
maxvit_default_fold_3-sammask
Model 'maxvit' loaded.
Checkpont Install Complete!! checkpoint_model: maxvit fold : 3 version: sammask
Metrics Save Complete!! maxvit_sammask_3.pkl
maxvit_default_fold_4-sammask
Model 'maxvit' loaded.
Checkpont Install Complete!! checkpoint_model: maxvit fold : 4 version: sammask
Metrics Save Complete!! maxvit_sammask_4.pkl
maxvit_default_fold_5-sammask
Model 'maxvit' loaded.
Checkpont Install Complete!! checkpoint_model: maxvit fold : 5 version: sammask
Metrics Save Complete!! maxvit_sammask_5.pkl
mobilenet_l_fold_1-sammask
Model 'mobilenet' loaded.
Checkpont Install Complete!! checkpoint_model: mobilenet fold : 1 version: sammask
Metrics Save Complete!! mob

# 전체 메트릭 비교 

In [6]:
import matplotlib.pyplot as plt
import numpy as np
import os
import re
import pandas as pd
import pickle

# metric_save 함수는 이전에 수정된 버전을 사용합니다.

def metric_save(save_metric_dir: str, pickles: list):
    # Initialize DataFrame to store results
    all_metrics = pd.DataFrame(columns=['Model', 'Version', 'Fold', 'Accuracy', 'Precision',
                                        'Sensitivity', 'Specificity', 'F1-score', 'AUC'])
    roc_metrics = {}
    # Read data from pickle files and add to DataFrame
    for idx, pk in enumerate(pickles):
        pickle_path = os.path.join(save_metric_dir, pk)
        # Use regex to extract model_name, version, fold_num
        match = re.match(r'(.+?)_(.+?)_(.+?)\.pkl$', pk)
        
        if match:
            model_name, version, fold_num = match.groups()
            if version == 'transformer':
                version = 'mask-ft-medsam1'
        else:
            print(f"Filename {pk} does not match the expected pattern. Skipping.")
            continue  # Skip files that don't match the expected pattern

        # Load data from pickle file
        with open(pickle_path, 'rb') as f:
            metric = pickle.load(f)

        roc_metrics[f'{model_name}_{version}_{fold_num}'] = {
            'fpr': metric['FPR'],
            'tpr': metric['TPR'],
            'roc_auc': metric['AUC'],
            'thresholds': metric['Thresholds']
        }
        # Convert metric data to DataFrame
        metric_df = pd.DataFrame(metric)

        # Select required columns and the last row
        metric_df = metric_df[['Accuracy', 'Precision', 'Sensitivity', 'Specificity', 'F1-score', 'AUC']].iloc[-1]

        # Add additional info: Model, Version, Fold
        metric_df['Model'] = model_name
        metric_df['Version'] = version
        metric_df['Fold'] = pd.to_numeric(fold_num, errors='coerce')  # Ensure 'Fold' is numeric

        # Reorder columns
        metric_df = metric_df[['Model', 'Version', 'Fold', 'Accuracy', 'Precision', 'Sensitivity', 'Specificity', 'F1-score', 'AUC']]

        # Append to all_metrics DataFrame
        all_metrics = pd.concat([all_metrics, metric_df.to_frame().T], ignore_index=True)

    # Exclude 'Fold' from mean calculation
    avg_metrics = all_metrics.drop(columns=['Fold']).groupby(['Model', 'Version'], as_index=False).mean()

    # Sort by AUC in descending order
    avg_metrics = avg_metrics.sort_values(by='AUC', ascending=False)

    # Round numeric columns to 3 decimal places
    numeric_cols = ['Accuracy', 'Precision', 'Sensitivity', 'Specificity', 'F1-score', 'AUC']
    avg_metrics[numeric_cols] = avg_metrics[numeric_cols].apply(pd.to_numeric, errors='coerce').round(3)

    return roc_metrics, all_metrics, avg_metrics

# 저장된 metric 파일이 있는 디렉토리 경로
save_metric_dir = '/mnt/hdd/octc/BACKUP/BreastUS/experiment/metrics'
# 디렉토리 내 모든 pickle 파일 목록을 정렬
pickles = sorted(os.listdir(save_metric_dir))

# metrics 데이터를 불러오는 함수
roc_metrics, all_metrics, avg_metrics = metric_save(save_metric_dir, pickles)
avg_metrics

,Model,Version,Accuracy,Precision,Sensitivity,Specificity,F1-score,AUC
25,swin-transformer,mask,0.898,0.835,0.856,0.918,0.845,0.948
0,convnext,mask,0.897,0.829,0.862,0.914,0.845,0.946
30,vision-transformer,mask,0.897,0.852,0.828,0.930,0.839,0.945
10,maxvit,mask,0.896,0.829,0.856,0.915,0.842,0.945
5,efficient,mask,0.889,0.823,0.840,0.913,0.831,0.937
20,resnet,mask,0.879,0.816,0.812,0.911,0.814,0.937
15,mobilenet,mask,0.872,0.795,0.817,0.899,0.806,0.925
9,efficient,sammask,0.857,0.780,0.784,0.893,0.781,0.911
29,swin-transformer,sammask,0.841,0.755,0.757,0.882,0.756,0.906
33,vision-transformer,sammask,0.851,0.785,0.752,0.898,0.765,0.905


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from lib.metric.visualize import *
# seaborn 스타일 설정
sns.set(style="whitegrid")

# 모델별로 'mask', 'origin', 그리고 'sammask' 성능 차이를 계산하는 함수
def calculate_performance_diff(df):
    model_diff = []

    # 모델별로 그룹핑하여 성능 차이를 계산
    for model in df['Model'].unique():
        origin_row = df[(df['Model'] == model) & (df['Version'] == 'origin')]
        mask_row = df[(df['Model'] == model) & (df['Version'] == 'mask')]
        sammask_row = df[(df['Model'] == model) & (df['Version'] == 'sammask')]
        
        # 'origin'과 'mask'의 성능 차이 계산
        if not mask_row.empty and not origin_row.empty:
            diff = mask_row.iloc[0][2:] - origin_row.iloc[0][2:]
            diff['Model'] = model
            diff['Comparison'] = 'mask_vs_origin'
            model_diff.append(diff)
        
        # 'origin'과 'sammask'의 성능 차이 계산
        if not sammask_row.empty and not origin_row.empty:
            diff_sam = sammask_row.iloc[0][2:] - origin_row.iloc[0][2:]
            diff_sam['Model'] = model
            diff_sam['Comparison'] = 'sammask_vs_origin'
            model_diff.append(diff_sam)
    
    return pd.DataFrame(model_diff)

# 모델별로 'mask', 'origin', 그리고 'sammask' 성능을 도넛형 차트로 시각화하는 함수
# def plot_donut_chart(df, metric):
#     for model in df['Model'].unique():
#         mask_row = df[(df['Model'] == model) & (df['Version'] == 'mask')].iloc[0]
#         origin_row = df[(df['Model'] == model) & (df['Version'] == 'origin')].iloc[0]
#         sammask_row = df[(df['Model'] == model) & (df['Version'] == 'sammask')].iloc[0]

#         mask_value = mask_row[metric]
#         origin_value = origin_row[metric]
#         sammask_value = sammask_row[metric]

#         # 도넛형 차트 그리기
#         fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(aspect="equal"))

#         # 각 성능의 값을 계산하고 라벨 지정
#         values = [mask_value, origin_value, sammask_value]
#         labels = ['Mask', 'Origin', 'SamMask']
#         colors = ['#66b3ff', '#ff9999', '#99ff99']

#         # 도넛형 차트 그리기
#         wedges, texts, autotexts = ax.pie(values, labels=labels, autopct='%.1f%%', startangle=90, colors=colors,
#                                           wedgeprops=dict(width=0.3, edgecolor='w'))

#         # 도넛형 가운데에 텍스트 추가
#         ax.text(0, 0, f'{model}', horizontalalignment='center', verticalalignment='center', fontsize=15, weight='bold')

#         # 제목 및 시각화 꾸미기
#         plt.setp(autotexts, size=12, weight="bold")
#         ax.set_title(f'{model} - {metric} Comparison (Mask vs Origin vs SamMask)', fontsize=14)
        
#         plt.show()

# # 바이올린 플롯 그리기 함수
# def plot_violin_plot(df, metric):
#     plt.figure(figsize=(10, 6))
    
#     # 바이올린 플롯 생성
#     sns.violinplot(x='Model', y=metric, hue='Version', data=df, split=True, inner="quart", palette="Set2")
    
#     # 플롯 꾸미기
#     plt.title(f'{metric} Comparison between Mask, Origin, and SamMask', fontsize=14)
#     plt.ylabel(f'{metric} Value')
#     plt.xlabel('Model')
#     plt.xticks(rotation=45)
#     plt.grid(True)
#     plt.show()

# 중요한 성능 지표들에 대해 바이올린 플롯 그리기
important_metrics = ['Accuracy', 'Precision', 'Sensitivity', 'Specificity', 'F1-score', 'AUC']

# # 바이올린 플롯과 도넛 차트 그리기
# for metric in important_metrics:
#     plot_violin_plot(avg_metrics, metric)
#     plot_donut_chart(avg_metrics, metric)

# 레이다 차트 그리기 (모델별로)
for model in avg_metrics['Model'].unique():
    plot_radar_chart(avg_metrics, model)


#  ETC 

In [ ]:
import pandas as pd 
pd.read_csv('/mnt/hdd/octc/experiment/binary.csv')

In [ ]:
import sys, os 
sys.path.append('../')
import torchvision.transforms as transforms
from lib.datasets.ds_tools import kfold_extract
from lib.dataset import Custom_stratified_Dataset
from torchsampler import ImbalancedDatasetSampler


from lib.datasets.sampler import class_weight_sampler, class_weight_getter
from torch.utils.data import DataLoader
folds, train_df, test_df  = kfold_extract('/mnt/hdd/octc/experiment/binary.csv',  5, True, random_state =627)
for idx, fold in enumerate(folds):
    print(f'Fold {idx+1}')
    # fold안에 레이블 분포 확인
    print(fold['train']['label'].value_counts())
    print(fold['val']['label'].value_counts())
    print('-------------------')
    dataset = Custom_stratified_Dataset(
        df = fold['train'],
        root_dir = '/mnt/hdd/octc/BACKUP/Dataset',
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.Grayscale(num_output_channels=3),
            transforms.ToTensor()
        ])
    )
    
    # sampler = class_weight_sampler(fold)
    
    train_loader = DataLoader(
        dataset,
        batch_size = 10,
        sampler = ImbalancedDatasetSampler(dataset),
        num_workers = 8
    )
    
    for img, label in train_loader:
        print(label.unique(return_counts=True))
    print('-------------------')
    # print(class_weight_getter(fold))
    
        
    
    
    print('-------------------')
    dataset
    
    
    
    break 

In [ ]:
import sys, os 
sys.path.append('../')
import torchvision.transforms as transforms
import numpy as np
import torch
device = 'cuda'
from lib.datasets.ds_tools import kfold_extract
from lib.dataset import Custom_stratified_Dataset
from lib.datasets.sampler import class_weight_sampler, class_weight_getter
from torch.utils.data import DataLoader
folds, train_df, test_df  = kfold_extract('/mnt/hdd/octc/experiment/binary.csv',  5, True, random_state =627)
for idx, fold in enumerate(folds):
    print(f'Fold {idx+1}')
    # fold안에 레이블 분포 확인
    print(fold['train']['label'].value_counts())
    print(fold['val']['label'].value_counts())
    print('-------------------')
    dataset = Custom_stratified_Dataset(
        df = fold['train'],
        root_dir = '/mnt/hdd/octc/BACKUP/Dataset',
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.Grayscale(num_output_channels=3),
            transforms.ToTensor()
        ])
    )
    
    n_pos = (fold['train']['label']==1).sum()
    n_neg = (fold['train']['label']==0).sum()
    pos_weight_value = n_neg / n_pos
    pos_weight = torch.tensor([pos_weight_value], dtype=torch.float32).to('cuda')

    
    print(pos_weight)
    print('-------------------')
    
    
    
    break 

In [ ]:
from sklearn.model_selection import GroupKFold, KFold
import pandas as pd
def k_fold_split(csv_path, random_seed =42 ):
    df = pd.read_csv(csv_path)
    groups = df['pid']  # Assuming 'pid' is the column for patient IDs
    y = df['label']     # Assuming 'label' is the target variable

    # folder = GroupKFold(n_splits=5)
    folder = KFold(n_splits=5, shuffle=True, random_state=random_seed)
    folds = []

    # for train_idx, val_idx in folder.split(df, y, groups):
    for train_idx, val_idx in folder.split(df, y, groups):
        train_df = df.iloc[train_idx].reset_index(drop=True)
        val_df = df.iloc[val_idx].reset_index(drop=True)
        fold = {'train': train_df, 'val': val_df}
        folds.append(fold)
    return folds

folds = k_fold_split(csv_path = '/mnt/hdd/octc/experiment/dataset-train.csv')
for fold in folds:
    # train과 valid에서 겹치는 pid 개수 확인
    print(len(set(fold['train']['pid'].unique()) & set(fold['val']['pid'].unique())))